# RFM Segmentation 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as mpatches
import seaborn as sns
import re
import warnings

In [2]:
df1 = pd.read_csv('/Users/rajathadri_as/Documents/Sep14_2023.csv')

C:\Users\rajathadri_as\AppData\Local\Temp\ipykernel_4080\3856658171.py:1: DtypeWarning: Columns (35,64,116,121,145,146,148,150,151,172,176,179,183,190,208,212,218,225,231,273,329) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('/Users/rajathadri_as/Documents/Sep14_2023.csv')


In [3]:
df = df1[['id','user_id','username','email','phone','type_id','type_name','package_id','active','suspended','approved','price','cost','num_photos','seller_type_id','trade','video','boost','boost_active','caotw','hppremium','views','business_url_click','contacts','displays','views_photo','url_click','print_click','print_phone','days_running','createdon']]

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20785 entries, 0 to 20784
Data columns (total 31 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  20785 non-null  int64 
 1   user_id             20785 non-null  int64 
 2   username            20785 non-null  object
 3   email               16904 non-null  object
 4   phone               17385 non-null  object
 5   type_id             20785 non-null  int64 
 6   type_name           20785 non-null  object
 7   package_id          20785 non-null  int64 
 8   active              20785 non-null  int64 
 9   suspended           20785 non-null  int64 
 10  approved            20785 non-null  int64 
 11  price               20785 non-null  int64 
 12  cost                20785 non-null  int64 
 13  num_photos          20785 non-null  int64 
 14  seller_type_id      20785 non-null  int64 
 15  trade               20785 non-null  int64 
 16  video               20

In [5]:
# Dataset Duplicate Value Count
len(df[df.duplicated()])

0

In [6]:
# Missing Values/Null Values Count
print(df.isnull().sum())

id                       0
user_id                  0
username                 0
email                 3881
phone                 3400
type_id                  0
type_name                0
package_id               0
active                   0
suspended                0
approved                 0
price                    0
cost                     0
num_photos               0
seller_type_id           0
trade                    0
video                    0
boost                    0
boost_active             0
caotw                    0
hppremium                0
views                    0
business_url_click       0
contacts                 0
displays                 0
views_photo              0
url_click                0
print_click              0
print_phone              0
days_running             0
createdon                0
dtype: int64


In [7]:
# Check Unique Values for each variable.
df.nunique()

id                    20785
user_id                3136
username               3136
email                  3161
phone                  3164
type_id                  10
type_name                10
package_id                9
active                    2
suspended                 2
approved                  3
price                  2926
cost                     57
num_photos               92
seller_type_id            2
trade                     2
video                     2
boost                     2
boost_active              2
caotw                     2
hppremium                 2
views                  1788
business_url_click        3
contacts                 48
displays              11524
views_photo            4071
url_click               188
print_click              79
print_phone              60
days_running             11
createdon             20285
dtype: int64

In [8]:
#Changing datatype : date => object -> datetime64
df = df.astype({
    'createdon':'datetime64[ns]'
})

In [9]:
df.head()

,id,user_id,username,email,phone,type_id,type_name,package_id,active,suspended,...,views,business_url_click,contacts,displays,views_photo,url_click,print_click,print_phone,days_running,createdon
0,1311488,200715,HighbrigeDevon,caravansdevon@highbridgecaravans.co.uk,01626 832792,7,Motorhomes,61,1,0,...,554,0,0,14806,1990,4,1,0,0,2023-03-05 11:40:21
1,1312256,158926,tynevalleymotorhome,sales@tynevalleymotorhomes.co.uk,01207 272777,7,Motorhomes,57,1,0,...,225,0,0,1273,1030,7,0,0,0,2023-03-08 11:40:12
2,1313024,158795,highbridgecaravans,info@highbridgecaravans.co.uk,01278 782725,7,Motorhomes,61,1,0,...,36,0,0,2562,273,3,4,0,0,2023-03-11 11:40:42
3,1315584,179544,westcountryhomes,sales@westcountrymotorhomes.co.uk,01278761200,7,Motorhomes,61,1,0,...,260,0,1,5948,863,2,0,0,0,2023-03-21 15:42:16
4,1316352,246530,CockerhamSands,martyn.turner@pureleisuregroup.com,01524 751387,2,Static Caravans,61,1,0,...,476,0,0,28518,2286,5,0,0,0,2023-03-23 07:40:08


# <div class = 'alert alert-info'>Recency</div>

In [10]:
df['createdon'].describe(include=all)

count                            20785
mean     2023-02-10 23:53:21.424056064
min                2015-06-26 15:14:51
25%                2023-01-25 20:41:20
50%                2023-05-11 10:41:53
75%                2023-06-23 10:40:16
max                2023-09-13 23:11:20
Name: createdon, dtype: object

In [11]:
# Creating a new dataframe that includes only the last transaction dates of each user
df_recency = df.groupby('user_id')['createdon'].max().reset_index()
df_recency.rename(columns={'createdon': 'latest_purchase_date'}, inplace=True)
df_recency.head()

,user_id,latest_purchase_date
0,5750,2023-09-12 08:41:01
1,6398,2020-09-10 19:15:02
2,6738,2023-03-26 15:01:29
3,6928,2023-08-23 20:37:16
4,7339,2023-08-18 13:31:08


In [12]:
df_recency['latest_purchase_date'].describe(include=all)

count                             3136
mean     2023-05-10 00:17:43.850765056
min                2016-07-17 07:32:10
25%                2023-03-30 11:47:45
50%                2023-06-20 18:26:57
75%                2023-08-14 11:33:17
max                2023-09-13 23:11:20
Name: latest_purchase_date, dtype: object

In [13]:
# Calculating the recency value
recent_date = df_recency['latest_purchase_date'].max()
df_recency['Recency'] = df_recency['latest_purchase_date'].apply(lambda x: (recent_date - x).days)
df_recency.head()

,user_id,latest_purchase_date,Recency
0,5750,2023-09-12 08:41:01,1
1,6398,2020-09-10 19:15:02,1098
2,6738,2023-03-26 15:01:29,171
3,6928,2023-08-23 20:37:16,21
4,7339,2023-08-18 13:31:08,26


In [14]:
df_recency.Recency.describe()

count    3136.000000
mean      126.574298
std       196.803796
min         0.000000
25%        30.000000
50%        85.000000
75%       167.000000
max      2614.000000
Name: Recency, dtype: float64

# <div class = 'alert alert-info'>Frequency</div>

In [15]:
# Creating new dataframe to calculate frequency of customers
frequency_df = df.groupby(by=['user_id'], as_index=False)['createdon'].count()
frequency_df.columns = ['user_id', 'Frequency']

In [16]:
frequency_df.head()

,user_id,Frequency
0,5750,45
1,6398,2
2,6738,1
3,6928,1
4,7339,2


In [17]:
# Frequency values
frequency_df['Frequency'].unique()

array([  45,    2,    1,    3,    7,   11,    6,    4,    5,  362,   29,
         81,   61,   71,   25,   22,   35,   48, 1183,  313,  369,   75,
         10,  564,   91,   26,  149,  140,   37,   36,  231,   78,  131,
        122,   44,   19,  249,  142,  100,   42,   38,   40,   76,   15,
        239,   68,   46,  145,   20,   74,  219,  227,   84,  316,   17,
         33,  160,   49,  205,    9,   18,   16,    8,   31,   14,   64,
        501,   13,  276,   51,  195,  390,   32,  137,   89,  312,  164,
         39,   53,  373,  207,   54,   30,  130,   21,   23,   85,  103,
         27,   28,   52,   12,   43,   73,   24, 3328,   66,  255],
      dtype=int64)

In [18]:
frequency_df.Frequency.describe()

count    3136.000000
mean        6.627870
std        68.821153
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
max      3328.000000
Name: Frequency, dtype: float64

# <div class = 'alert alert-info'>Monetary</div>

In [19]:
monetary_df = df.groupby(by='user_id', as_index=False)['cost'].sum()
monetary_df.columns = ['user_id', 'Monetary']
monetary_df['Monetary'] = monetary_df['Monetary']/100
monetary_df.head()

,user_id,Monetary
0,5750,0.0
1,6398,0.0
2,6738,0.0
3,6928,0.0
4,7339,0.0


In [20]:
monetary_df.Monetary.unique()

array([  0.  ,  12.99,  35.98,   8.99,  17.99,  19.99,  27.99,  15.99,
        19.58,   6.99,  11.98,  37.98,  12.98,  15.98,  10.99,  16.98,
        30.98,  25.99,  14.99,  13.98,  10.49,  46.97,   9.99,  12.59,
       103.96,  24.98,  42.97,   7.99,  24.99,  22.98,  33.98,  29.98,
        12.49,  32.98,  19.98,   4.99,  28.98,  17.98,  23.99,  36.98,
        31.97,  49.98,  26.99,  53.97, 493.81,  39.99,   5.99,  50.99,
        40.99,  26.98,  27.98,   4.89,  27.96,  19.59,  18.98,  45.98,
        32.97,  11.88,  40.97,  20.98,  29.97,  22.97,  13.99,  29.99,
         6.49,  47.98,  18.19,  45.97,  23.98,  40.98,   6.29,  39.98])

In [21]:
monetary_df.Monetary.describe()

count    3136.000000
mean        3.859554
std        12.013891
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max       493.810000
Name: Monetary, dtype: float64

# <div class = 'alert alert-danger'> RFM Dictionary</div>

In [22]:
# merging dataframes on 'user_id' one after the other
rf_df = df_recency.merge(frequency_df, on='user_id')
rfm_df = rf_df.merge(monetary_df, on='user_id')

In [23]:
rfm_df.head()

,user_id,latest_purchase_date,Recency,Frequency,Monetary
0,5750,2023-09-12 08:41:01,1,45,0.0
1,6398,2020-09-10 19:15:02,1098,2,0.0
2,6738,2023-03-26 15:01:29,171,1,0.0
3,6928,2023-08-23 20:37:16,21,1,0.0
4,7339,2023-08-18 13:31:08,26,2,0.0


In [24]:
rfm_df.shape

(3136, 5)

In [25]:
rfm_df.user_id.nunique()

3136

In [32]:
# Creating ranks by sorting
rfm_df['R_rank'] = rfm_df['Recency'].rank(ascending=False)   
rfm_df['F_rank'] = rfm_df['Frequency'].rank(ascending=True)
rfm_df['M_rank'] = rfm_df['Monetary'].rank(ascending=True)
 
# normalizing the rank of the customers
# by dividing from the highest value in the ranks
rfm_df['R_rank_norm'] = (rfm_df['R_rank']/rfm_df['R_rank'].max())*100
rfm_df['F_rank_norm'] = (rfm_df['F_rank']/rfm_df['F_rank'].max())*100
rfm_df['M_rank_norm'] = (rfm_df['M_rank']/rfm_df['M_rank'].max())*100

In [33]:
# Dropping non-normalized RFM features
rfm_df.drop(columns=['R_rank', 'F_rank', 'M_rank'], inplace=True)

rfm_df.head()

,user_id,latest_purchase_date,Recency,Frequency,Monetary,R_rank_norm,F_rank_norm,M_rank_norm
0,5750,2023-09-12 08:41:01,1,45,0.0,98.730924,97.751913,38.839286
1,6398,2020-09-10 19:15:02,1098,2,0.0,0.803213,83.816964,38.839286
2,6738,2023-03-26 15:01:29,171,1,0.0,23.951807,39.732143,38.839286
3,6928,2023-08-23 20:37:16,21,1,0.0,82.538153,39.732143,38.839286
4,7339,2023-08-18 13:31:08,26,2,0.0,78.361446,83.816964,38.839286


> (0.15 x Recency score) + (0.28 x Frequency score) + (0.57 x Monetary score)

In [34]:
# Applying the formula
rfm_df['RFM_Score'] = 0.15*rfm_df['R_rank_norm']+0.28 *rfm_df['F_rank_norm']+0.57*rfm_df['M_rank_norm']

In [35]:
# Calculating RFM out of 5 
rfm_df['RFM_Score'] *= 0.05

# Rounding to 2 decimal points
rfm_df = rfm_df.round(2)

In [36]:
# Dropping non-normalized RFM features
rfm_df.drop(columns=['R_rank_norm', 'F_rank_norm', 'M_rank_norm'], inplace=True)

rfm_df.head()

,user_id,latest_purchase_date,Recency,Frequency,Monetary,RFM_Score
0,5750,2023-09-12 08:41:01,1,45,0.0,3.22
1,6398,2020-09-10 19:15:02,1098,2,0.0,2.29
2,6738,2023-03-26 15:01:29,171,1,0.0,1.84
3,6928,2023-08-23 20:37:16,21,1,0.0,2.28
4,7339,2023-08-18 13:31:08,26,2,0.0,2.87


In [37]:
df_user_info = df[['user_id','username','email','phone']]

In [38]:
df_user_info.shape

(20785, 4)

In [41]:
rfm_dict = df_user_info.merge(rfm_df, on='user_id',how='left').drop_duplicates(keep='first').reset_index(drop=True)
rfm_dict

,user_id,username,email,phone,latest_purchase_date,Recency,Frequency,Monetary,RFM_Score
0,200715,HighbrigeDevon,caravansdevon@highbridgecaravans.co.uk,01626 832792,2023-07-29 15:01:07,46,373,0.0,3.01
1,158926,tynevalleymotorhome,sales@tynevalleymotorhomes.co.uk,01207 272777,2023-07-29 14:57:07,46,231,0.0,3.00
2,158795,highbridgecaravans,info@highbridgecaravans.co.uk,01278 782725,2023-07-29 15:01:37,46,1183,0.0,3.01
3,179544,westcountryhomes,sales@westcountrymotorhomes.co.uk,01278761200,2023-07-29 15:17:23,46,501,0.0,3.01
4,246530,CockerhamSands,martyn.turner@pureleisuregroup.com,01524 751387,2023-03-23 07:40:08,174,12,0.0,2.61
...,...,...,...,...,...,...,...,...,...
3292,246932,traceysjones,traceysjones@btinternet.com,07748655428,2023-02-25 10:35:01,200,1,0.0,1.77
3293,250374,andywiffen,andywiffen@hotmail.co.uk,07780838251,2023-06-30 13:30:07,75,1,0.0,2.07
3294,40058,nands,nands@nandsnet.co.uk,07525779058,2023-01-22 17:19:02,234,1,0.0,1.72
3295,243313,ingetp,ingetp@btinternet.com,07889 66 88 52,2023-09-13 13:40:36,0,1,0.0,2.41


In [42]:
rfm_dict= rfm_dict.groupby(['user_id'])[['username','email','phone','latest_purchase_date','Recency','Frequency','Monetary','RFM_Score']].first().reset_index()

rfm_dict.head()

,user_id,username,email,phone,latest_purchase_date,Recency,Frequency,Monetary,RFM_Score
0,5750,wilkinsmotorcaravans,sales@wilkinsmotorcaravans.co.uk,01285 821216,2023-09-12 08:41:01,1,45,0.0,3.22
1,6398,chris64,sales@parkleisureuk.co.uk,01834 891444,2020-09-10 19:15:02,1098,2,0.0,2.29
2,6738,lindpetewinn,lindpetewinn@hotmail.com,07840229183,2023-03-26 15:01:29,171,1,0.0,1.84
3,6928,YI91295G6152801,msteersy@yahoo.com,07818 426393,2023-08-23 20:37:16,21,1,0.0,2.28
4,7339,YI913G7H4160212,kerry.emery@spar.co.uk,07976962805,2023-08-18 13:31:08,26,2,0.0,2.87


In [43]:
rfm_dict.shape

(3136, 9)

In [44]:
rfm_dict.user_id.nunique()

3136

In [45]:
rfm_dict.isnull().sum()

user_id                  0
username                 0
email                    6
phone                   50
latest_purchase_date     0
Recency                  0
Frequency                0
Monetary                 0
RFM_Score                0
dtype: int64

In [46]:
# Group by 'user_id' and count the number of ads posted by each user
user_ads_count = df.groupby(['user_id','type_name'])['id'].count().reset_index()
user_ads_count.columns = ['user_id', 'type_name', 'ads_count']

# Filter users who have posted at least 3 ads
user_ads_count = user_ads_count[user_ads_count['ads_count'] >= 3]

In [47]:
user_ads_count.sort_values(by= 'user_id')

,user_id,type_name,ads_count
1,5750,Motorhomes,44
17,28328,Accessories,3
37,32040,Accessories,7
41,33202,Accessories,11
49,34270,Accessories,3
...,...,...,...
3309,257627,Motorhomes,90
3308,257627,Campervans,36
3310,257627,Touring Caravans,129
3315,257642,Motorhomes,8


In [48]:
user_ads_count['user_id'].nunique()

346

In [49]:
result_df = pd.merge(user_ads_count, rfm_dict, on='user_id', how='left')

In [50]:
result_df.head()

,user_id,type_name,ads_count,username,email,phone,latest_purchase_date,Recency,Frequency,Monetary,RFM_Score
0,5750,Motorhomes,44,wilkinsmotorcaravans,sales@wilkinsmotorcaravans.co.uk,01285 821216,2023-09-12 08:41:01,1,45,0.0,3.22
1,28328,Accessories,3,peter-irene.stanger,peter-irene.stanger@blueyonder.co.uk,01226211522,2023-08-19 16:36:51,25,3,0.0,2.95
2,32040,Accessories,7,petedobbie,petedobbie@live.co.uk,07442188857,2023-05-04 15:06:38,132,7,0.0,2.70
3,33202,Accessories,11,riverside55,riverside55@talk21.com,07759606464,2023-08-31 07:49:07,13,11,0.0,3.10
4,34270,Accessories,3,maggie.francis,maggie.francis@talktalk.net,01291629206,2023-09-09 15:09:44,4,3,0.0,3.08


In [51]:
result_df.to_csv('users_rfm.csv')